In [1]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
from tqdm import tqdm


# Download the dataset from Kaggle (assuming you've downloaded it beforehand)
# Replace 'path/to/sunbird-english-prompts.csv' with the actual path
data_path = '/kaggle/input/sunbird-english-prompts/english_prompts.csv'
df = pd.read_csv(data_path)

# Function to select the first two words of each sentence
def get_first_two_words(text):
    words = text.split()
    return ' '.join(words[:2]) if len(words) >= 2 else text  # Handle cases with less than 2 words

# Apply the function to the 'text' column
df['first_two_words'] = df['Text'].apply(get_first_two_words)

# Load the TinyLlama tokenizer and model
torch.manual_seed(0)
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model_name = "facebook/opt-350m"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)

# Function to generate text using the LLM
def generate_text(prompt, device):
    input_ids = tokenizer.encode(prompt, return_tensors='pt').to(device)
    output = model.generate(input_ids, max_length=50)  # Adjust max_length as needed
    return tokenizer.decode(output[0], skip_special_tokens=True)

# Generate text for the first two words of each sentence and store in a list
generated_texts = []
for idx, row in tqdm(df.iterrows(), total=len(df)):  # Use tqdm for progress bar
    prompt = row['first_two_words']
    generated_text = generate_text(prompt, device)
    generated_texts.append(generated_text)

# Create a new DataFrame with the generated text and save as a CSV file
df_output = pd.DataFrame({'Generated Text': generated_texts})
df_output.to_csv('generated_texts.csv', index=False)


tokenizer_config.json:   0%|          | 0.00/685 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/644 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/441 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/663M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

100%|██████████| 25006/25006 [5:37:38<00:00,  1.23it/s]
